### Imports and setting CONSTANTS


In [1]:
import torch.nn as nn
from torch.autograd import Variable
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)

torch.cuda.empty_cache()

NUM_TOKENS = 50257
EMBEDDING_SIZE = 512
HIDDEN_SIZE = 512
TARGET_SIZE = 2
BATCH_SIZE = 64
MODEL_ID = "projecte-aina/FLOR-6.3B"
NUM_LAYERS = 512
MAX_LENGTH = 512
# NUM_TOKENS = 10000
# EMBEDDING_SIZE = 300
# HIDDEN_SIZE = 300
# TARGET_SIZE = 2
# BATCH_SIZE = 20
# MODEL_ID = "projecte-aina/FLOR-6.3B"
# NUM_LAYERS = 107
# MAX_LENGTH = 107

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/tmp/ipykernel_6214/3255591236.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Custom LSTM Neural network

In [9]:
class CustomEmbedding(nn.Module):
  def __init__(self, encoder, input_size, embedding_size, hidden_size, target_size):
        
        super(CustomEmbedding, self).__init__()
        
        self.embedding = nn.Embedding(num_embeddings=input_size, embedding_dim=embedding_size)   
        self.encoder = encoder
        self.lastlayer = nn.Linear(hidden_size, target_size)
      
       


  def forward(self, sentence):
      embeds = self.embedding(sentence)
      trans_logit = self.encoder(embeds)
      
      return self.lastlayer(trans_logit)
    

### Custom Dataset

In [3]:
class MoviesDataset(Dataset):
    def __init__(self, x, y, model_max_length):
        self.x = x
        self.y = y
        self.model_max_length = model_max_length

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        text = self.x[idx]
        text_tokenized = tokenizer.encode(text, padding='max_length', max_length=self.model_max_length , truncation=True, return_tensors='pt', )
        y_hot = nn.functional.one_hot(torch.tensor(self.y[idx]), num_classes=2)
        
        return text_tokenized[0], y_hot

### Training

In [4]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)

    for batch, (X, y) in enumerate(dataloader):
        # X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)


        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    return loss

### Prediction

In [5]:
def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            # X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### Main method starting point

In [10]:
def main(batch_size, input_size, embedding_size, hidden_size, target_size, num_layers, epochs=None):
    dataset = pd.read_csv(
        "../datasets/IMDB-Dataset.csv", usecols=["review", "sentiment"]
    )
    dataset["sentiment"] = dataset["sentiment"].map({"positive": 1, "negative": 0})
    train, test = train_test_split(dataset, test_size=0.2)
    learning_rate = 0.003

    encoder = nn.Transformer(d_model=512, nhead=8, dim_feedforward=2048, dropout=0.1, activation="gelu", custom_encoder=None, custom_decoder=None, layer_norm_eps=1e-05, batch_first=False, norm_first=False, bias=True, device=None, dtype=None).encoder.layers[0]
    model = CustomEmbedding(encoder, input_size, embedding_size, hidden_size, target_size)
    

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    train_dataset = MoviesDataset(train['review'].values, train['sentiment'].values, model_max_length=num_layers)
    test_dataset = MoviesDataset(test['review'].values, test['sentiment'].values, model_max_length=num_layers)

    
    train_dataloader = DataLoader(train_dataset, batch_size , shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size, shuffle=True)

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")

        train_loop(train_dataloader, model, loss_fn, optimizer)
        test_loop(test_dataloader, model, loss_fn)


main(
    batch_size=BATCH_SIZE,
    input_size=50257,
    embedding_size=EMBEDDING_SIZE,
    hidden_size=HIDDEN_SIZE,
    target_size=TARGET_SIZE,
    num_layers=NUM_LAYERS,
    epochs=10,
)

/home/adria/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1
-------------------------------
loss: 4.662898  [    0/40000]
loss: 1.115108  [  640/40000]
loss: 1.657282  [ 1280/40000]
loss: 5.726794  [ 1920/40000]
loss: 0.938670  [ 2560/40000]
loss: 1.151015  [ 3200/40000]
loss: 0.789729  [ 3840/40000]
loss: 0.790114  [ 4480/40000]
loss: 0.950367  [ 5120/40000]
loss: 0.864568  [ 5760/40000]
loss: 0.750104  [ 6400/40000]
loss: 0.960788  [ 7040/40000]
loss: 0.994710  [ 7680/40000]
loss: 0.792030  [ 8320/40000]
loss: 0.750696  [ 8960/40000]
loss: 0.722606  [ 9600/40000]
loss: 0.766440  [10240/40000]
loss: 0.838112  [10880/40000]
loss: 0.969817  [11520/40000]
loss: 0.895237  [12160/40000]
loss: 0.733892  [12800/40000]
loss: 1.065675  [13440/40000]
loss: 0.756095  [14080/40000]
loss: 1.049976  [14720/40000]
loss: 0.860166  [15360/40000]
loss: 1.136674  [16000/40000]
loss: 1.069410  [16640/40000]
loss: 0.802097  [17280/40000]
loss: 0.768728  [17920/40000]
loss: 0.732851  [18560/40000]
loss: 0.927380  [19200/40000]
loss: 0.726935  [19840/40000]


KeyboardInterrupt: 